[Same model pretrain + finetune  ](https://colab.research.google.com/drive/1lbqiSiA0sDF7JDWPeS0tccrM85LloVha?usp=sharing#scrollTo=n4WLPzYlutg6)

# ลองเอา model pretrain มาpredictเลย eiei

In [ ]:
import pandas as pd
from transformers import BlipProcessor, BlipForConditionalGeneration
import requests
from PIL import Image
from tqdm import tqdm


In [47]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")
try:
    df = pd.read_csv('predict_this_img.csv')
except FileNotFoundError:
    print("Error: CSV file 'predict_this_img.csv' not found. Please ensure it exists in the correct location.")
    exit()


selected_columns = df[['coco_url', 'id']]
selected_columns.head()

,coco_url,id
0,http://images.cocodataset.org/val2017/00000046...,463174
1,http://images.cocodataset.org/val2017/00000046...,463199
2,http://images.cocodataset.org/val2017/00000046...,463283
3,http://images.cocodataset.org/val2017/00000046...,463522
4,http://images.cocodataset.org/val2017/00000046...,463527


In [42]:
results = []
for i in tqdm(range(len(selected_columns)), desc="Processing Images"):
    try:
        raw_image = Image.open(requests.get(selected_columns.iloc[i]['coco_url'], stream=True).raw).convert('RGB')


        inputs = processor(raw_image, return_tensors="pt").to("cuda")


        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)

        results.append({'id': selected_columns.iloc[i]['id'], 'caption': caption})
    except Exception as e:
        print(f"Error processing image {i+1}: {e}")
        continue  r

results_df = pd.DataFrame(results)


results_df.to_csv('resultshaah.csv', index=False)




Processing Images:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Processing Images: 100%|██████████| 1000/1000 [09:47<00:00,  1.70it/s]

Results successfully saved to resultshaah.csv
